# Model dataset

## Cargamos los dataset preprocesados:

---> [Notebook de exploración y submuestreo](dataset_exploration.ipynb)

In [1]:
import pandas as pd

# Definir rutas
train_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_preprocessed.pkl"
test_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_preprocessed.pkl"
sample_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_sample_preprocessed.pkl"
train_set_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_set.pkl"
val_set_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/val_set.pkl"

# Cargar los datasets
train = pd.read_pickle(train_path)
test = pd.read_pickle(test_path)
train_sample = pd.read_pickle(sample_path)
train_set = pd.read_pickle(train_set_path)
val_set = pd.read_pickle(val_set_path)

# Confirmar que los datos se han cargado correctamente
print(f"Train: {train.shape}")
print(f"Test: {test.shape}")
print(f"Train sample: {train_sample.shape}")
print(f"Train set: {train_set.shape}")
print(f"Validation set: {val_set.shape}")


Train: (36858893, 9)
Test: (29275, 8)
Train sample: (368589, 9)
Train set: (294871, 9)
Validation set: (73718, 9)


---

## Análisis de columnas comunes

Para proceder con el enriquecimiento, primero confirmemos las columnas comunes entre Train y Test:

In [2]:
# Identificar columnas comunes entre Train y Test
common_columns = set(train.columns).intersection(set(test.columns))
print(f"Columnas comunes entre Train y Test: {common_columns}")


Columnas comunes entre Train y Test: {'timestamp_local', 'user_id', 'pagetype', 'device_type', 'partnumber', 'session_id', 'country', 'date'}


### **Análisis de las columnas comunes**

#### **Columnas comunes identificadas:**
1. **`country`**: Representa el país asociado a la interacción.
   - **Potencial**: Segmentación geográfica para recomendaciones basadas en tendencias locales.
2. **`user_id`**: Identificador único del usuario (puede ser `-1` para no logueados).
   - **Potencial**: Diferenciación de estrategias para usuarios logueados y no logueados.
3. **`session_id`**: Identificador único de la sesión.
   - **Potencial**: Agrupación y análisis por sesión.
4. **`date`**: Fecha de la interacción.
   - **Potencial**: Análisis temporal, detección de patrones de compra por días o periodos.
5. **`device_type`**: Tipo de dispositivo utilizado.
   - **Potencial**: Personalización basada en dispositivo.
6. **`pagetype`**: Tipo de página visitada.
   - **Potencial**: Contexto de la interacción (e.g., página de producto vs. categoría).
7. **`partnumber`**: Identificador único del producto.
   - **Potencial**: Relación directa con el producto interactuado.
8. **`timestamp_local`**: Marca temporal detallada.
   - **Potencial**: Análisis de comportamiento dentro de sesiones.

### **Propuestas de enriquecimiento**

1. **Enriquecimiento temporal**:
   - Crear nuevas características basadas en `date` y `timestamp_local`:
     - Día de la semana, franja horaria (mañana, tarde, noche).
     - Tiempo desde la última interacción dentro de una sesión.

2. **Agrupaciones por `session_id` y `user_id`**:
   - Número total de interacciones en la sesión.
   - Diversidad de productos interactuados en una sesión.

3. **Segmentación geográfica (`country`)**:
   - Análisis de productos más populares por país.
   - Crear un indicador de "producto popular en este país" para `partnumber`.

4. **Segmentación por `device_type` y `pagetype`**:
   - Identificar tendencias de interacción según el tipo de dispositivo.
   - Detectar patrones específicos en páginas de producto o categorías.

---

## Enriquecimiento de los datasets
Para alinear las propuestas de enriquecimiento con nuestro enfoque **híbrido** (colaborativo, basado en contenido y popularidad) y los **4 tipos de usuarios**, avancemos con un plan que priorice cada técnica según su relevancia.

### **Plan de enriquecimiento alineado al enfoque**

1. **Colaborativo**:
   - **Agrupaciones por `user_id` y `session_id`**:
     - Contar interacciones por sesión (`session_length`).
     - Diversidad de productos interactuados por sesión (`unique_products_count`).
     - Popularidad del producto (`product_popularity`), basado en la frecuencia de interacción en `train`.

2. **Basado en contenido**:
   - **Relación producto-sesión**:
     - Indicador de productos similares dentro de una sesión (`similar_products_count`).
   - **Tiempo de interacción**:
     - Añadir características derivadas de `timestamp_local`:
       - Franja horaria de interacción (`time_of_day`).
       - Diferencia entre interacciones consecutivas (`time_diff`).

3. **Popularidad**:
   - **Segmentación geográfica (`country`)**:
     - Crear un indicador para "producto popular en este país" (`country_popularity`).
   - **Dispositivo (`device_type`)**:
     - Detectar tendencias de popularidad según el dispositivo.

---

### **Enriquecimiento colaborativo**

#### 1. Número de interacciones por sesión (session_length)

In [3]:
# Calcular el número de interacciones por sesión
train['session_length'] = train.groupby('session_id')['session_id'].transform('count')
test['session_length'] = test.groupby('session_id')['session_id'].transform('count')

#### 2. Diversidad de productos interactuados (unique_products_count)

In [4]:
# Calcular la cantidad de productos únicos por sesión
train['unique_products_count'] = train.groupby('session_id')['partnumber'].transform('nunique')
test['unique_products_count'] = test.groupby('session_id')['partnumber'].transform('nunique')


#### 3. Popularidad del producto (product_popularity)

In [5]:
# Calcular la frecuencia de interacción de cada producto en Train
product_popularity = train['partnumber'].value_counts().to_dict()

# Mapear la popularidad de los productos en Train y Test
train['product_popularity'] = train['partnumber'].map(product_popularity)
test['product_popularity'] = test['partnumber'].map(product_popularity)


#### Validación de enriquecimientos

In [6]:
# Revisar las primeras filas para confirmar los enriquecimientos
print(train[['session_id', 'session_length', 'unique_products_count', 'product_popularity']].head())
print(test[['session_id', 'session_length', 'unique_products_count', 'product_popularity']].head())


          session_id  session_length  unique_products_count  \
10725642     4696487              40                     40   
35628113     4572779              69                     69   
35628118     4572779              69                     69   
9475628      3239459             110                    110   
9475555      3239459             110                    110   

          product_popularity  
10725642                1001  
35628113                 510  
35628118                 593  
9475628                 6896  
9475555                 5398  
   session_id  session_length  unique_products_count  product_popularity
0         746               7                      7              5439.0
1         746               7                      7             19686.0
2         746               7                      7              1823.0
3         746               7                      7              9457.0
4         746               7                      7                12

#### Análisis de los resultados de los enriquecimientos colaborativos

##### **1. `session_length`**
- **Train**:
  - Ejemplo: `session_id = 4696487` tiene `session_length = 40`, lo que indica 40 interacciones en esta sesión.
- **Test**:
  - Ejemplo: `session_id = 746` tiene `session_length = 7`, consistente con el número de interacciones en las filas de este `session_id`.

**Conclusión**: 
- La columna `session_length` está correctamente calculada en ambos datasets y es coherente con el agrupamiento por `session_id`.

---

##### **2. `unique_products_count`**
- **Train**:
  - Ejemplo: `session_id = 4696487` tiene `unique_products_count = 40`, igual a `session_length`. Esto implica que todas las interacciones son con productos distintos en esta sesión.
- **Test**:
  - Ejemplo: `session_id = 746` tiene `unique_products_count = 7`, igual a `session_length`. Todas las interacciones son con productos diferentes.

**Conclusión**:
- La columna `unique_products_count` refleja correctamente la diversidad de productos por sesión.

---

##### **3. `product_popularity`**
- **Train**:
  - Ejemplo: `partnumber = 1001` tiene una popularidad de `1001`, lo que indica que este producto fue interactuado 1001 veces en el dataset de entrenamiento.
- **Test**:
  - Ejemplo: `partnumber = 5439` tiene una popularidad de `5439` mapeada correctamente desde el `train`.

**Observación en `Test`**:
- Algunos productos (`partnumber`) tendrán valores nulos en `product_popularity` si no están presentes en `train`. Esto es esperable, ya que `Test` puede incluir productos nuevos.

In [7]:
# Manejar valores nulos en product_popularity de Test
test['product_popularity'] = test['product_popularity'].fillna(1)

---

### **Enriquecimientos basados en contenido**

1. `similar_products_count`

    Cálculo:
    - Determinamos la cantidad de veces que un producto específico es interactuado en la misma sesión.
    - Esto puede indicar si un producto es especialmente relevante dentro de esa sesión.

In [8]:
# Calcular número de veces que un producto específico es interactuado en la misma sesión
train['similar_products_count'] = train.groupby(['session_id', 'partnumber'])['partnumber'].transform('count')
test['similar_products_count'] = test.groupby(['session_id', 'partnumber'])['partnumber'].transform('count')


2. `time_of_day`
    Cálculo:

    Convertimos timestamp_local a una franja horaria categórica:
    - Mañana: 06:00 - 12:00
    - Tarde: 12:00 - 18:00
    - Noche: 18:00 - 00:00
    - Madrugada: 00:00 - 06:00

In [9]:
# Convertir timestamp_local a tipo datetime si es necesario
train['timestamp_local'] = pd.to_datetime(train['timestamp_local'])
test['timestamp_local'] = pd.to_datetime(test['timestamp_local'])

In [10]:
# Crear función para asignar franjas horarias
def assign_time_of_day(timestamp):
    hour = timestamp.hour
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'

# Aplicar la función al dataset
# Aplicar la función al dataset
train['time_of_day'] = train['timestamp_local'].apply(assign_time_of_day)
test['time_of_day'] = test['timestamp_local'].apply(assign_time_of_day)



3. `time_diff`
    Cálculo:

    - Diferencia de tiempo entre interacciones consecutivas dentro de la misma sesión.

In [11]:
# Calcular diferencia de tiempo entre interacciones consecutivas en la misma sesión
train['time_diff'] = train.groupby('session_id')['timestamp_local'].diff().dt.total_seconds()
test['time_diff'] = test.groupby('session_id')['timestamp_local'].diff().dt.total_seconds()

# Llenar valores faltantes con un valor específico (e.g., 0)
train['time_diff'] = train['time_diff'].fillna(0)
test['time_diff'] = test['time_diff'].fillna(0)


In [13]:
# Verificar las primeras filas para confirmar la franja horaria
print(train[['timestamp_local', 'time_of_day']].head())
print(test[['timestamp_local', 'time_of_day']].head())

                 timestamp_local time_of_day
10725642 2024-06-15 11:02:27.710     morning
35628113 2024-06-15 03:43:20.699       night
35628118 2024-06-15 03:43:08.240       night
9475628  2024-06-15 03:37:24.111       night
9475555  2024-06-15 03:37:03.935       night
          timestamp_local time_of_day
0 2024-06-15 18:36:47.390     evening
1 2024-06-15 18:37:04.052     evening
2 2024-06-15 18:37:48.159     evening
3 2024-06-15 18:38:19.899     evening
4 2024-06-15 18:38:46.492     evening


#### Validación

Observación: 

En este caso, similar_products_count = 1 podría indicar que la característica no tiene variabilidad. Podría ser útil validar si hay sesiones donde el mismo producto aparece repetido para confirmar su utilidad.

In [14]:
# Revisar si hay productos repetidos en una misma sesión
train.groupby(['session_id', 'partnumber']).size().value_counts()


1    36858893
Name: count, dtype: int64

Observación:

Los valores negativos en time_diff en Train sugieren que las interacciones no están ordenadas por tiempo dentro de las sesiones. Esto puede ser solucionado ordenando los datos antes de calcular la diferencia.

In [15]:
# Reordenar por sesión y tiempo antes de calcular time_diff
train = train.sort_values(by=['session_id', 'timestamp_local'])
train['time_diff'] = train.groupby('session_id')['timestamp_local'].diff().dt.total_seconds().fillna(0)

test = test.sort_values(by=['session_id', 'timestamp_local'])
test['time_diff'] = test.groupby('session_id')['timestamp_local'].diff().dt.total_seconds().fillna(0)


In [16]:
# Validar las nuevas columnas en Train
print(train[['session_id', 'similar_products_count', 'time_of_day', 'time_diff']].head())

# Validar las nuevas columnas en Test
print(test[['session_id', 'similar_products_count', 'time_of_day', 'time_diff']].head())


          session_id  similar_products_count time_of_day  time_diff
16169047           1                       1   afternoon      0.000
35654420           3                       1     evening      0.000
35654419           3                       1     evening     11.399
17808629           4                       1   afternoon      0.000
17808628           4                       1   afternoon      3.042
   session_id  similar_products_count time_of_day  time_diff
0         746                       1     evening      0.000
1         746                       1     evening     16.662
2         746                       1     evening     44.107
3         746                       1     evening     31.740
4         746                       1     evening     26.593


#### **Análisis**


**1. `similar_products_count`**
- **Resultado**:
  - Todos los valores de `similar_products_count` siguen siendo **1** para todas las filas de `Train` y `Test`.
  - Esto indica que no hay sesiones donde el mismo producto haya sido interactuado más de una vez.

**Conclusión**:
- **`similar_products_count` no aporta variabilidad** en este caso. Por tanto, puede no ser una característica útil para el modelo, y es razonable descartarla para no introducir ruido.

---

**2. `time_of_day`**
- **Train**:
  - Ejemplo: `session_id = 16169047` tiene un valor de **`afternoon`** que coincide con la hora de la interacción.
  - Todos los valores están correctamente calculados y siguen reflejando las franjas horarias asignadas.
- **Test**:
  - Similarmente, los valores como **`evening`** son coherentes con los horarios en `timestamp_local`.

**Conclusión**:
- **`time_of_day` está validada y lista para usarse.**

---
**3. `time_diff`**
- **Train**:
  - Ejemplo: `session_id = 16169047` tiene un `time_diff = 0.000` para la primera interacción, mientras que las siguientes tienen valores como `11.399` (segundos entre interacciones consecutivas).
  - Los valores ahora son positivos tras ordenar los datos por `session_id` y `timestamp_local`, lo que asegura consistencia.
- **Test**:
  - Ejemplo: `session_id = 746` tiene diferencias de tiempo como `16.662`, `44.107`, lo que también refleja correctamente los intervalos entre interacciones.

**Conclusión**:
- **`time_diff` está correctamente calculada** y puede ser una característica valiosa para modelar comportamientos dentro de una sesión.

#### Descartamos `similar_products_count` en ambos datasets

In [17]:
# Eliminar similar_products_count de Train y Test
train.drop(columns=['similar_products_count'], inplace=True)
test.drop(columns=['similar_products_count'], inplace=True)

# Confirmar que la columna ha sido eliminada
print("Columnas en Train después de eliminar similar_products_count:")
print(train.columns)
print("\nColumnas en Test después de eliminar similar_products_count:")
print(test.columns)


Columnas en Train después de eliminar similar_products_count:
Index(['session_id', 'date', 'timestamp_local', 'add_to_cart', 'user_id',
       'country', 'partnumber', 'device_type', 'pagetype', 'session_length',
       'unique_products_count', 'product_popularity', 'time_of_day',
       'time_diff'],
      dtype='object')

Columnas en Test después de eliminar similar_products_count:
Index(['session_id', 'date', 'timestamp_local', 'user_id', 'country',
       'partnumber', 'device_type', 'pagetype', 'session_length',
       'unique_products_count', 'product_popularity', 'time_of_day',
       'time_diff'],
      dtype='object')


---

### **Enriquecimientos basados en popularidad**
Estos enriquecimientos están diseñados para capturar tendencias globales o segmentadas en los datos, particularmente útiles para usuarios nuevos y usuarios no logueados.

#### **Propuestas de enriquecimiento**
- **`country_popularity`**:
   - Calcular la popularidad de cada producto en función del país (`country`).
   - Utilizar la frecuencia de interacción en el `train` agrupada por `country` y `partnumber`.

- **`device_popularity`**:
   - Calcular la popularidad de cada producto en función del dispositivo (`device_type`).
   - Similar al cálculo de popularidad por país, pero segmentado por dispositivo.



1. `country_popularity`

In [18]:
# Calcular la popularidad de cada producto por país en Train
country_popularity = train.groupby(['country', 'partnumber']).size().reset_index(name='country_popularity')

# Mapear la popularidad al Train y Test
train = train.merge(country_popularity, on=['country', 'partnumber'], how='left')
test = test.merge(country_popularity, on=['country', 'partnumber'], how='left')

# Llenar valores faltantes en Test con 0
test['country_popularity'] = test['country_popularity'].fillna(0)


2. `device_popularity`

In [19]:
# Calcular la popularidad de cada producto por dispositivo en Train
device_popularity = train.groupby(['device_type', 'partnumber']).size().reset_index(name='device_popularity')

# Mapear la popularidad al Train y Test
train = train.merge(device_popularity, on=['device_type', 'partnumber'], how='left')
test = test.merge(device_popularity, on=['device_type', 'partnumber'], how='left')

# Llenar valores faltantes en Test con 0
test['device_popularity'] = test['device_popularity'].fillna(0)


#### Validación

In [20]:
# Validar nuevas columnas en Train
print(train[['country', 'partnumber', 'country_popularity', 'device_type', 'device_popularity']].head())

# Validar nuevas columnas en Test
print(test[['country', 'partnumber', 'country_popularity', 'device_type', 'device_popularity']].head())


   country  partnumber  country_popularity  device_type  device_popularity
0       57       31751                 204            1                623
1       29       35349                 497            1               1964
2       29        7701                 399            1               1481
3       57        5092                1579            1               4800
4       57       15276                2609            1              22711
   country  partnumber  country_popularity  device_type  device_popularity
0       57        1254              1474.0            1             5201.0
1       57       32544              5340.0            1            18906.0
2       57       12639               225.0            1             1666.0
3       57       18048              1593.0            1             8966.0
4       57       13295                 4.0            1               11.0


#### Análisis

##### **1. `country_popularity`**
- **Train**:
  - Ejemplo: Para `country = 57` y `partnumber = 31751`, la popularidad es **204**. Esto refleja la cantidad de veces que este producto fue interactuado en este país dentro del conjunto de entrenamiento.
  - Otros valores como **1579** (`partnumber = 5092`) y **2609** (`partnumber = 15276`) indican productos más populares en este país.

- **Test**:
  - Los valores como **1474.0** (`partnumber = 1254`) y **5340.0** (`partnumber = 32544`) reflejan las mismas métricas de popularidad basadas en el mapeo desde `Train`.

##### Observaciones:
- Los valores se asignan correctamente en `Test` para productos presentes en `Train`.
- Para productos nuevos (no presentes en `Train`), la popularidad sería **0**.

---

##### **2. `device_popularity`**
- **Train**:
  - Ejemplo: Para `device_type = 1` y `partnumber = 31751`, la popularidad es **623**. Esto indica cuántas veces este producto fue interactuado desde este tipo de dispositivo.
  - Valores más altos, como **22711** (`partnumber = 15276`), sugieren productos más populares para este tipo de dispositivo.

- **Test**:
  - Los valores mapeados, como **5201.0** (`partnumber = 1254`) y **18906.0** (`partnumber = 32544`), son coherentes con las interacciones en `Train`.

##### Observaciones:
- Al igual que con `country_popularity`, los valores se asignan correctamente en `Test`, y los productos nuevos tienen una popularidad predeterminada de **0**.

---

##### **Conclusión de los enriquecimientos basados en popularidad**
1. **`country_popularity`**:
   - Calculada y mapeada correctamente.
   - Captura tendencias de popularidad a nivel geográfico, útil para recomendaciones para usuarios no logueados.

2. **`device_popularity`**:
   - Calculada y mapeada correctamente.
   - Útil para capturar patrones específicos de dispositivos, como móviles vs. escritorio.

3. **Productos nuevos en `Test`**:
   - La asignación de popularidad predeterminada de **0** es adecuada, ya que refleja que no hay datos previos en `Train`.

---

## Persistencia de dataset enriquecidos basados en la estratégia del modelo de recomendaciones

In [21]:
# Definir rutas para guardar los datasets enriquecidos
train_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_enriched_model.pkl"
test_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_enriched_model.pkl"

# Guardar los datasets enriquecidos
train.to_pickle(train_enriched_path)
test.to_pickle(test_enriched_path)

print("Datasets enriquecidos guardados correctamente.")


Datasets enriquecidos guardados correctamente.


---